<a href="https://colab.research.google.com/github/Niirav213/Early-warning-for-critical-events-Predict-things-like-sepsis-or-heart-failure-before-they-happen/blob/main/sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from google.colab import files

print("Upload training_setA folder (zipped)")
uploaded_A = files.upload()  # Upload your training_setA.zip
print("Upload training_setB folder (zipped)")
uploaded_B = files.upload()  # Upload your training_setB.zip

# Unzip
import zipfile

# Get the actual filename from the uploaded dictionary
zip_file_A_name = list(uploaded_A.keys())[0]
zip_file_B_name = list(uploaded_B.keys())[0]


with zipfile.ZipFile(zip_file_A_name, 'r') as zip_ref:
    zip_ref.extractall("training_setA")

with zipfile.ZipFile(zip_file_B_name, 'r') as zip_ref:
    zip_ref.extractall("training_setB")

# List files in the extracted directories to help identify the correct path
#print("\nFiles in training_setA:")
#for root, dirs, files in os.walk("training_setA"):
 #   for name in files:
      #  print(os.path.join(root, name))

#print("\nFiles in training_setB:")
#for root, dirs, files in os.walk("training_setB"):
 #   for name in files:
 #       print(os.path.join(root, name))




Upload training_setA folder (zipped)


Saving training.zip to training.zip
Upload training_setB folder (zipped)


Saving training_setB.zip to training_setB.zip


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# =========================
# BUILD PREPROCESSING PIPELINE
# =========================
def build_pipeline(df, target="SepsisLabel", missing_threshold=0.95):
    """
    Build and fit preprocessing pipeline on a GLOBAL dataframe.
    """

    # 1. Drop highly missing features (GLOBAL missingness)
    missing_frac = df.isnull().mean()
    features_to_drop = missing_frac[missing_frac > missing_threshold].index.tolist()
    df_reduced = df.drop(columns=features_to_drop)

    # 2. Separate numeric and categorical features
    num_features = (
        df_reduced
        .select_dtypes(include=["float64", "int64"])
        .drop(columns=[target])
        .columns
        .tolist()
    )

    cat_features = ["Gender", "Unit1", "Unit2"]
    cat_features = [c for c in cat_features if c in df_reduced.columns]
    num_features = [c for c in num_features if c not in cat_features]

    # 3. Transformers
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    # 4. ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_features),
            ("cat", categorical_transformer, cat_features)
        ]
    )

    # 5. Final pipeline
    pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

    # Fit pipeline on GLOBAL data
    X = df_reduced.drop(columns=[target])
    pipeline.fit(X)

    return pipeline


# =========================
# APPLY PIPELINE
# =========================
def apply_pipeline(df, pipeline, target="SepsisLabel"):
    X = df.drop(columns=[target])
    y = df[target]

    X_preprocessed = pipeline.transform(X)
    feature_names = pipeline.named_steps["preprocessor"].get_feature_names_out()

    X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=feature_names)

    return X_preprocessed_df, y


# =========================
# LOAD GLOBAL SAMPLE (KEY FIX)
# =========================
def load_global_sample(input_folder, max_files=200):
    """
    Load data from multiple patient files to estimate GLOBAL missingness.
    Prevents dropping labs absent in a single patient.
    """
    dfs = []
    count = 0

    for root, _, files in os.walk(input_folder):
        for f in files:
            if f.endswith(".psv"):
                df = pd.read_csv(os.path.join(root, f), sep="|")
                dfs.append(df)
                count += 1

                if count >= max_files:
                    return pd.concat(dfs, ignore_index=True)

    return pd.concat(dfs, ignore_index=True)


# =========================
# PROCESS DATASET FOLDER
# =========================
def main(input_folder, output_folder, target="SepsisLabel"):
    os.makedirs(output_folder, exist_ok=True)

    # Find all PSV files
    psv_files = []
    for root, _, files in os.walk(input_folder):
        for f in files:
            if f.endswith(".psv"):
                psv_files.append(os.path.join(root, f))

    if not psv_files:
        print(f"No PSV files found in {input_folder}")
        return

    print(f"Found {len(psv_files)} PSV files in {input_folder}")

    # 🔑 GLOBAL PIPELINE FIT (FIX)
    print("Building global sample for preprocessing...")
    global_df = load_global_sample(input_folder, max_files=200)
    print(f"Global sample shape: {global_df.shape}")

    pipeline = build_pipeline(global_df, target=target)

    # Process each patient file
    for file_path in psv_files:
        filename = os.path.basename(file_path)
        print(f"Processing {filename}...")

        df = pd.read_csv(file_path, sep="|")
        X_preprocessed, y = apply_pipeline(df, pipeline, target=target)

        df_out = pd.concat(
            [X_preprocessed, y.reset_index(drop=True)],
            axis=1
        )

        output_path = os.path.join(output_folder, filename)
        df_out.to_csv(output_path, sep="|", index=False)

    print(f"✅ Finished processing {input_folder}")


# =========================
# ENTRY POINT
# =========================
if __name__ == "__main__":

    # TRAINING (SET A)
    print("Processing training_setA...")
    main(
        input_folder="training_setA",
        output_folder="processed_training"
    )

    # VALIDATION / TEST (SET B)
    print("Processing training_setB...")
    main(
        input_folder="training_setB",
        output_folder="processed_training_setB"
    )


Streaming output truncated to the last 5000 lines.
Processing p117756.psv...
Processing p109488.psv...
Processing p101883.psv...
Processing p110705.psv...
Processing p113857.psv...
Processing p105757.psv...
Processing p102913.psv...
Processing p114488.psv...
Processing p116026.psv...
Processing p117069.psv...
Processing p100440.psv...
Processing p107050.psv...
Processing p103542.psv...
Processing p106675.psv...
Processing p117410.psv...
Processing p118001.psv...
Processing p110838.psv...
Processing p116717.psv...
Processing p110498.psv...
Processing p109566.psv...
Processing p101858.psv...
Processing p118736.psv...
Processing p112736.psv...
Processing p103651.psv...
Processing p102313.psv...
Processing p109730.psv...
Processing p107580.psv...
Processing p101017.psv...
Processing p112698.psv...
Processing p112692.psv...
Processing p115423.psv...
Processing p107819.psv...
Processing p105903.psv...
Processing p114535.psv...
Processing p115466.psv...
Processing p108920.psv...
Processing p1

In [3]:
import os
import pandas as pd
import numpy as np


In [4]:
import os
import pandas as pd

all_columns = set()

for f in os.listdir("processed_training"):
    if f.endswith(".psv"):
        df = pd.read_csv(os.path.join("processed_training", f), sep="|", nrows=1)
        all_columns.update(df.columns)

FEATURE_COLUMNS = sorted(c for c in all_columns if c != "SepsisLabel")

print("Final feature count:", len(FEATURE_COLUMNS))
print(FEATURE_COLUMNS)


Final feature count: 32
['cat__Gender_0.0', 'cat__Gender_1.0', 'cat__Unit1_0.0', 'cat__Unit1_1.0', 'cat__Unit2_0.0', 'cat__Unit2_1.0', 'num__Age', 'num__BUN', 'num__BaseExcess', 'num__Chloride', 'num__Creatinine', 'num__DBP', 'num__FiO2', 'num__Glucose', 'num__HCO3', 'num__HR', 'num__Hct', 'num__Hgb', 'num__HospAdmTime', 'num__ICULOS', 'num__MAP', 'num__Magnesium', 'num__O2Sat', 'num__PTT', 'num__PaCO2', 'num__Platelets', 'num__Potassium', 'num__Resp', 'num__SBP', 'num__Temp', 'num__WBC', 'num__pH']


In [5]:
def load_patient_sequences(folder, feature_columns, early_hours=6):
    X, y = [], []

    for f in os.listdir(folder):
        df = pd.read_csv(os.path.join(folder, f), sep="|")

        # 🔑 FORCE SAME FEATURE SPACE
        df = df.reindex(columns=feature_columns + ["SepsisLabel"])
        df[feature_columns] = df[feature_columns].fillna(0.0)

        features = df[feature_columns].values
        labels = df["SepsisLabel"].values

        if labels.max() == 1:
            t_sepsis = np.where(labels == 1)[0][0]
            cutoff = t_sepsis - early_hours
            if cutoff <= 0:
                continue
            seq = features[:cutoff]
            label = 1
        else:
            seq = features
            label = 0

        X.append(seq)
        y.append(label)

    return X, np.array(y)


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [7]:
def pad_data(X, max_len=None):
    if max_len is None:
        max_len = max(len(x) for x in X)

    X_pad = pad_sequences(
        X,
        maxlen=max_len,
        dtype="float32",
        padding="pre",
        truncating="pre"
    )

    return X_pad, max_len


In [9]:
# TRAIN (Set A)
X_train_raw, y_train = load_patient_sequences("processed_training", FEATURE_COLUMNS)
X_train, max_len = pad_data(X_train_raw)

# VALIDATION / TEST (Set B)
X_val_raw, y_val = load_patient_sequences("processed_training_setB", FEATURE_COLUMNS)
X_val, _ = pad_data(X_val_raw, max_len=max_len)


In [10]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)

TIMESTEPS = X_train.shape[1]
N_FEATURES = X_train.shape[2]

print("TIMESTEPS:", TIMESTEPS)
print("N_FEATURES:", N_FEATURES)

assert N_FEATURES == 32


X_train shape: (19900, 336, 32)
X_val shape: (19658, 336, 32)
TIMESTEPS: 336
N_FEATURES: 32


In [11]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0,1]),
    y=y_train
)

class_weight = {0: class_weights[0], 1: class_weights[1]}
print(class_weight)


{0: np.float64(0.536503828318775), 1: np.float64(7.348596750369277)}


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

def build_lstm(units=64, dropout=0.3):
    model = Sequential([
        Input(shape=(TIMESTEPS, N_FEATURES)),
        LSTM(units, return_sequences=False),
        Dropout(dropout),
        Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc")]
    )

    return model


In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    "units": [32, 64],
    "dropout": [0.3, 0.5]
}

best_auc = 0.0
best_model = None

for params in ParameterGrid(param_grid):
    print("Training with:", params)

    model = build_lstm(
        units=params["units"],
        dropout=params["dropout"]
    )

    model.fit(
        X_train,
        y_train,
        epochs=10,
        batch_size=32,
        class_weight=class_weight,
        validation_data=(X_val, y_val),
        verbose=1
    )

    val_auc = model.evaluate(X_val, y_val, verbose=0)[1]

    if val_auc > best_auc:
        best_auc = val_auc
        best_model = model


Training with: {'dropout': 0.3, 'units': 32}
Epoch 1/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 72s 110ms/step - auc: 0.7561 - loss: 0.5602 - val_auc: 0.7982 - val_loss: 0.6667
Epoch 2/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 78s 104ms/step - auc: 0.9037 - loss: 0.3787 - val_auc: 0.7942 - val_loss: 0.6792
Epoch 3/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 90s 117ms/step - auc: 0.9275 - loss: 0.3383 - val_auc: 0.7981 - val_loss: 0.7247
Epoch 4/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 64s 103ms/step - auc: 0.9230 - loss: 0.3338 - val_auc: 0.7784 - val_loss: 0.9467
Epoch 5/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 64s 102ms/step - auc: 0.9424 - loss: 0.3074 - val_auc: 0.7800 - val_loss: 0.8731
Epoch 6/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 89s 114ms/step - auc: 0.9494 - loss: 0.2764 - val_auc: 0.7994 - val_loss: 0.7240
Epoch 7/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 67s 108ms/step - auc: 0.9561 - loss: 0.2630 - val_auc: 0.7870 - val_loss: 0.9024
Epoch 8/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 64s 103ms/step - auc: 0.9556 - loss: 0.2674 - val_auc: 0.7989 - val_

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
from sklearn.metrics import f1_score

y_prob = best_model.predict(X_val).ravel()

thresholds = np.linspace(0.1, 0.9, 50)
best_t, best_f1 = 0, 0

for t in thresholds:
    y_pred = (y_prob > t).astype(int)
    f1 = f1_score(y_val, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best threshold:", best_t)
print("Best F1:", best_f1)


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
y_pred = (y_prob > best_t).astype(int)

print("ROC-AUC:", roc_auc_score(y_val, y_prob))
print(classification_report(y_val, y_pred))


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pos_weight = torch.tensor([scale_pos_weight]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

NameError: name 'scale_pos_weight' is not defined